# Checkpointing in a Training Loop TODO(b/409382939) add links

This guide covers the usage of the `training` module, designed around the basic concept of a training loop.

In [ ]:
from orbax.checkpoint import v1 as ocp

training = ocp.training

## Getting Started

Let's dive in with a simple training loop example.

We will use the `Checkpointer` API provided by the `training` module. The `Checkpointer` must be configured with a **root directory**, which represents a working directory where all checkpoints will be saved throughout the course of an experiment.

The root directory is not itself a checkpoint; rather, it is a *container* of checkpoints.

In [ ]:
root_directory = '/tmp/my-checkpoints'

We will assume the existence of a training state containing the keys `params` and `opt_state`, which are trees of `jax.Array`. The state also contains a key `step`, which is represented as an integer.

Note that the arrays in the state will be sharded using a fully-replicated sharding, but the example would work equally well with any other sharding.

In [ ]:
import numpy as np
import jax

pytree = {
    'params': {
        'layer0': np.arange(16).reshape((8, 2)),
    },
    'opt_state': [np.arange(16)],
}
sharding = jax.sharding.NamedSharding(jax.sharding.Mesh(jax.devices(), ('x',)), jax.sharding.PartitionSpec())
pytree = jax.tree.map(lambda x: jax.device_put(x, sharding), pytree)
pytree['step'] = 0

Let's set up our fake training loop. We will add a "training step function" that just increments the step. In reality, this would also compute gradients and update model parameters.

In [ ]:
def train_step(state):
  state['step'] += 1
  return state

Now, we can create a `Checkpointer` to begin saving a sequence of checkpoints.

In [ ]:
with training.Checkpointer(root_directory) as ckptr:
  num_steps = 10
  for step in range(num_steps):
    saved = ckptr.save_pytree(step, pytree)
    assert saved
    pytree = train_step(pytree)

Calling `load` with no arguments will automatically restore the latest saved checkpoint.

In [ ]:
with training.Checkpointer(root_directory) as ckptr:
  print(ckptr.load_pytree())

## Checkpointer APIs

Now, let's get into a bit more detail about how to interact with the `Checkpointer`.

In general, we recommend using `Checkpointer` as a context manager, as shown in the examples below.

```
with Checkpointer(...) as ckptr:
  ...
```

You can use it without the context manager, but make sure to call `close()` before the program exits to ensure the completion of any outstanding operations and to ensure resource cleanup.

```
ckptr = Checkpointer(...)
...
ckptr.close()
```

### Saving

Calling `save` in the training loop automatically calls `should_save`, which determines whether or not a checkpoint should be saved at the given step, based on the configured saving frequency. If a save is performed `save` returns `True`; otherwise it returns `False`.

Whether or not a save should be performed can be controlled via `SaveDecisionPolicy`.

By default, `ContinuousCheckpointingPolicy` is configured, which always saves *unless* a save is already ongoing.

Other pre-configured policies include:
- `FixedIntervalPolicy`: Saves every `n` steps.
- `InitialSavePolicy`: Saves on the first step.
- `PreemptionCheckpointingPolicy`: Saves on a step where a preemption signal is received by the JAX distributed system. This is useful for saving whenever a job is automatically restarted by the system.
- `SpecificStepsPolicy`: Saves on the specific set of configured steps.

The policies can be used in conjunction via `AnySavePolicy`, which performs a save if any of the sub-policies would perform a save at the given step.

You may always implement your own policy. See `SaveDecisionPolicy` for details.

In [ ]:
root_directory = '/tmp/my-checkpoints-1'
with training.Checkpointer(root_directory, save_decision_policy=training.save_decision_policies.FixedIntervalPolicy(3)) as ckptr:
  for step in range(10):
    ckptr.save_pytree(step, pytree)

In [ ]:
!ls {root_directory}

Now let's exercise some additional save features. These include:

- `custom_metadata`: A JSON-formatted object intended for storing any user-specified properties. Custom metadata can be specified at both the root directory level and the individual checkpoint level. At the root level, the metadata should pertain to all checkpoints. For example, the experiment name is shared by all checkpoints within the root directory, while a property like `is_final` has different values for different checkpoints.
-  `force`: Performs a save at the current step regardless of what would ordinarily be dictated by the `SaveDecisionPolicy`.
- `metrics`: A JSON-formatted object storing evaluation metrics for the current step. This can be useful for ordering and garbage collecting checkpoints; more on that below.

In [ ]:
root_directory = '/tmp/my-checkpoints-2'
with training.Checkpointer(root_directory, save_decision_policy=training.save_decision_policies.FixedIntervalPolicy(3), custom_metadata={'experiment_name': 'my-experiment'}) as ckptr:
  num_steps = 10
  for step in range(num_steps):
    is_final = step == num_steps - 1
    ckptr.save_pytree(
        step,
        pytree,
        metrics={'accuracy': 0.85},
        custom_metadata={'is_final': is_final},
        force=is_final,
    )

In [ ]:
!ls {root_directory}

We will learn more about how to access some of the attributes that we saved in the sections below.

### Querying Available Checkpoints

We can learn about which checkpoints are available by using `latest` and `checkpoints`.

In [ ]:
ckptr = training.Checkpointer(root_directory)

Each of these APIs returns `CheckpointMetadata` objects, which store a number of properties describing each checkpoint. Some metadata properties are more expensive to retrieve than others though. The `latest` and `checkpoints` APIs just store a limited set of cheaply-retrievable properties, like the `step`. These APIs also make use of caching as much as possible, to avoid repeated disk reads.

In [ ]:
# Returns CheckpointMetadata or None, if no checkpoints are found.
latest = ckptr.latest
assert latest is not None
print(latest.step)
print(latest)

### Inspecting Checkpoint Metadata

In many cases, we wish to cheaply gain information about checkpoint properties without loading the entire model. Using the `pytree_metadata` API, we can learn about the tree structure of our PyTree, as well as information about each array in the tree.

Like loading methods, metadata methods accept either no argument, or an argument representing the step to retrieve metadata for.

For example:

In [ ]:
# Loads metadata from the latest checkpoint.
ckptr.pytree_metadata()
# Loads metadata corresponding to the first step.
ckptr.pytree_metadata(ckptr.checkpoints[0])
# Loads metadata from a specific integer step.
ckptr.pytree_metadata(3)

print()

Let's examine the output.

In [ ]:
ckptr.pytree_metadata()

Let's dig into a few specific fields. In particular, we can access `custom_metadata` and `metrics` that were saved previously.

In [ ]:
print(ckptr.pytree_metadata().metrics)
print(ckptr.pytree_metadata().custom_metadata)

Within the metadata object, there is another field called `metadata`. This stores information specific to the structure of the object we saved. In this case, it describes the structure of the PyTree and array properties.

In [ ]:
import pprint
pprint.pprint(ckptr.pytree_metadata().metadata)

Finally, we can also retrieve the root-level metadata. Recall that this metadata is intended to describe the entire sequence of checkpoints, rather than just a single checkpoint.

In [ ]:
ckptr.root_metadata()

### Loading

As we saw above with the `metadata` methods, we can load in a variety of ways.

In [ ]:
# Loads from the latest checkpoint.
ckptr.load_pytree()
# Loads the first available checkpoint in the root directory.
ckptr.load_pytree(ckptr.checkpoints[0])
# Loads from a specific integer step.
ckptr.load_pytree(3)

print()

When dealing with PyTrees, particularly PyTrees with sharded `jax.Array` leaves, it is important for any non-toy use cases to specify an "abstract PyTree" that is used to guide restoration. Checkpoints are complicated objects. The abstract PyTree acts as an assertion to verify that the checkpoint has structure you expect and that arrays have the correct shapes.

The abstract PyTree can also be used to instruct Orbax how to load the PyTree. The `dtype` property may be used to cast arrays, while the `sharding` property is used to correctly place array shards on devices.

We should define an abstract tree with the same structure as the tree we originally saved. For the leaves, we specify different shardings than we originally saved with, and different dtypes as well, causing the loaded arrays to be cast and resharded when loading.


In [ ]:
sharding = jax.sharding.NamedSharding(jax.sharding.Mesh(jax.devices(), ('x',)), jax.sharding.PartitionSpec('x'))
abstract_pytree = {
    'params': {
        'layer0': jax.ShapeDtypeStruct((8, 2), np.float32, sharding=sharding),
    },
    'opt_state': [jax.ShapeDtypeStruct((16,), np.float32, sharding=sharding)],
    'step': 0,
}

In [ ]:
ckptr.load_pytree(None, abstract_pytree)

More details on working with PyTrees in such a manner can be found at TODO(link).

### Checkpointables TODO(link)

`Checkpointer` supports the concept of `checkpointables`. See the documentation on "Working with Checkpointables" for more information.

In simplified terms, a "checkpointable" refers to a distinct piece of the overall checkpoint, which can be thought of as a bundle. The `PyTree` training state is one such checkpointable. The dataset iterator is another. Checkpointing the position of the dataset iterator can be useful to ensure training resumes where we were interrupted not just for the model parameters, but for the data as well.

We can see this concept in concrete terms using a Grain dataset iterator. See [Grain documentation](https://google-grain.readthedocs.io/en/latest/index.html) for more information. For our purposes, we can construct a toy dataset iterator.

In [ ]:
import grain

dataset = iter(
    grain.MapDataset.range(30)
    .batch(3)
    .map(lambda x: x.tolist())
)

pytree = {
    'params': {
        'layer0': np.arange(16).reshape((8, 2)),
    },
    'opt_state': [np.arange(16)],
}
sharding = jax.sharding.NamedSharding(jax.sharding.Mesh(jax.devices(), ('x',)), jax.sharding.PartitionSpec())
pytree = jax.tree.map(lambda x: jax.device_put(x, sharding), pytree)
pytree['step'] = 0

In [ ]:
def train_step(state, ds):
  next(ds)  # Advances the dataset iterator
  state['step'] += 1
  return state

We can save ten checkpoints in sequence, including the dataset iterator, advancing the iterator once per step. At each step, the dataset iterator points to `[step*3, step*3+1, step*3+2]`.

In [ ]:
root_directory = '/tmp/my-checkpoints-3'
num_steps = 10

with training.Checkpointer(root_directory) as ckptr:
  for step in range(num_steps):
    ckptr.save_checkpointables(
        step,
        dict(pytree=pytree, dataset=dataset)
    )
    pytree = train_step(pytree, dataset)

After loading at step `5`, `new_dataset` points to position `5` of the iterator.

In [ ]:
new_dataset = iter(
    grain.MapDataset.range(30)
    .batch(3)
    .map(lambda x: x.tolist())
)
print(f'Initial position: {next(new_dataset)}')

with training.Checkpointer(root_directory) as ckptr:
  ckptr.load_checkpointables(
      5,
      dict(pytree=None, dataset=new_dataset)
  )
print(f'Loaded from checkpoint: {next(new_dataset)}')

It's important to note that dataset loading is stateful. You need to instantiate an iterator object, pass it to `load_checkpointables`, and the checkpoint state will be restored into the iterator state of the dataset object.